In [1]:
import torch
from torch import nn

In [2]:
# 互相关运算
# X是张量，K是卷积核,Y是卷积结果
def corr2d(X, K):
    h, w = K.shape
    # Y的shape计算公式
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
# 卷积层网络
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        # 卷积核
        self.weight = nn.Parameter(torch.rand(kernel_size))
        # 偏置
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [5]:
net = Conv2D((2, 2))
net(X)

tensor([[ 7.1848, 10.1204],
        [15.9916, 18.9273]], grad_fn=<AddBackward0>)

In [6]:
"""
实例：利用卷积核进行边缘检测
通过找到像素变化的位置，来检测图像中不同颜⾊的边缘。⾸先，构造⼀个6 × 8像素的⿊⽩图像。中间四列为⿊⾊（0），其余像素为⽩⾊（1）
"""
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
"""
接下来，构造⼀个⾼度为1、宽度为2的卷积核K。当进⾏互相关运算时，如果⽔平相邻的两元素相同，则输出为零，否则输出为⾮零
"""
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [8]:
"""
问题：将输⼊的⼆维图像转置，再进⾏如上的互相关运算。其输出如下，之前检测到的垂直边缘消失了。这个卷积核K只可以检测垂直边缘，⽆法检测⽔平边缘
"""
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [9]:
# 学习卷积核
# 构造⼀个⼆维卷积层，它具有1个输出通道和形状为（1， 2）的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

In [10]:
# 这个⼆维卷积层使⽤四维输⼊和输出格式（批量⼤⼩、通道、⾼度、宽度），
# 其中批量⼤⼩和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
X, Y

(tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.]]]]),
 tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]]))

In [11]:
lr = 3e-2  # 学习率
for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

epoch 2, loss 10.602
epoch 4, loss 1.899
epoch 6, loss 0.368
epoch 8, loss 0.082
epoch 10, loss 0.022


In [12]:
# 查看学习到的卷积核参数，非常接近1，-1
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9713, -0.9957]])

In [13]:
# 填充
"""
应⽤多层卷积时，常常丢失边缘像素，解决方案是填充填充是在输⼊图像的边界填充元素。
如下创建⼀个高度和宽度为3的⼆维卷积层，并在所有侧边填充1个像素。
给定高度和宽度为8的输⼊，则输出的高度和宽度也是8。
如果没有填充操作：输出的卷积结果应该是(8-3+1)*(8-3+1)=6*6
"""

# 不填充
conv2d = nn.Conv2d(1, 1, kernel_size=3)
# 所有侧边填充1个像素
conv2d_padding = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8)).reshape((1, 1, 8, 8))
conv2d(X).shape, conv2d_padding(X).shape

(torch.Size([1, 1, 6, 6]), torch.Size([1, 1, 8, 8]))

In [14]:
# 步幅
"""
在计算互相关时，卷积窗⼝从输⼊张量的左上⻆开始，向下、向右滑动。在前⾯的例⼦中，我们默认每次滑动
⼀个元素。但是，有时候为了⾼效计算或是缩减采样次数，卷积窗⼝可以跳过中间位置，每次滑动多个元素。
"""
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
conv2d(X).shape

torch.Size([1, 1, 4, 4])

In [15]:
# 管道
"""
图像有RGB三个通道颜色，对多通道进行卷积运算，是把每个子通道卷积的结果结合
"""
# 一个两通道的数据
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
# 针对两个通道数据的卷积核
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
X.shape, K.shape

(torch.Size([2, 3, 3]), torch.Size([2, 2, 2]))

In [16]:
# 接收多输入管道数据，输出单管道，即把每个子通道卷积的结果结合
def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在⼀起
    return sum(corr2d(x, k) for x, k in zip(X, K))


corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [17]:
# 多输出通道
print(K.shape)
print(K)

torch.Size([2, 2, 2])
tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])


In [18]:
K = torch.stack((K, K + 1, K + 2), dim=0)
print(K)

tensor([[[[0., 1.],
          [2., 3.]],

         [[1., 2.],
          [3., 4.]]],


        [[[1., 2.],
          [3., 4.]],

         [[2., 3.],
          [4., 5.]]],


        [[[2., 3.],
          [4., 5.]],

         [[3., 4.],
          [5., 6.]]]])


In [19]:
print(K.shape)

torch.Size([3, 2, 2, 2])


In [20]:
# 多通道运算函数
# 对于X分别用K中不同通道的卷积核对X运算
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输⼊“X”执⾏互相关运算。
    # 最后将所有结果都叠加在⼀起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [21]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [22]:
# 池化层（汇聚层）
# 目的：降低卷积层对位置的敏感性，同时降低对空间降采样表⽰的敏感性
# 操作：如窗口取最大值或者平均值
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

In [25]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
print(X)
pool2d(X, (2, 2),mode="max"),pool2d(X, (2, 2),mode="avg")

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])


(tensor([[4., 5.],
         [7., 8.]]),
 tensor([[2., 3.],
         [5., 6.]]))